In [1]:
import pandas as pd
import numpy as np
import os.path

from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

from flyyufelix_models.resnet_101 import resnet101_model
from flyyufelix_models.custom_layers.scale_layer import Scale

import threading
lock = threading.Lock()

from keras.callbacks import ModelCheckpoint

from BsonIterator import BSONIterator

Using TensorFlow backend.


In [2]:
METADATA_DIR = 'metadata'
DATA_DIR = '../../dataset'

train_offsets_df = pd.read_csv(os.path.join(METADATA_DIR,"train_offsets.csv"), index_col=0)
train_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Train_0.5_0.1.csv"), index_col=0)
val_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Val_0.5_0.1.csv"), index_col=0)

num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 32

# Create a generator for training and a generator for validation.
# Tip: use ImageDataGenerator for data augmentation and preprocessing.
assert os.path.exists(os.path.join(DATA_DIR, 'train.bson'))
train_bson_file = open(os.path.join(DATA_DIR, 'train.bson'), "rb")
train_datagen = ImageDataGenerator()
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df,
                         num_classes, train_datagen, lock, target_size = (224,224),
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock, target_size = (224,224),
                       batch_size=batch_size, shuffle=True)

/home/ankdesh/virtualenvs/cdiscount-py2.7/local/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Found 5570369 images belonging to 5270 classes.
Found 614245 images belonging to 5270 classes.


In [3]:
from keras.callbacks import ModelCheckpoint

# Save model Dir
MODEL_DIR = 'saved_models'
IMAGE_ROWS = 224
IMAGE_COLS = 224
CHANNELS =3

# Checkpointer 
checkpointer = ModelCheckpoint(filepath='saved_models/Resnet101.{epoch:02d}-{acc:.2f}-{val_acc:.2f}_RandomSplit_Train_0.5_0.1.h5', verbose=1)

In [4]:
model = load_model('saved_models/Resnet101.01-0.55-0.55_RandomSplit_Train_0.5_0.1.h5',custom_objects={
                   'Scale':Scale})

/home/ankdesh/virtualenvs/cdiscount-py2.7/local/lib/python2.7/site-packages/keras/engine/topology.py:1252: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [8]:
for i,layer in enumerate(model.layers):
    print (i,layer.name)

(0, u'data')
(1, u'conv1_zeropadding')
(2, u'conv1')
(3, u'bn_conv1')
(4, u'scale_conv1')
(5, u'conv1_relu')
(6, u'pool1')
(7, u'res2a_branch2a')
(8, u'bn2a_branch2a')
(9, u'scale2a_branch2a')
(10, u'res2a_branch2a_relu')
(11, u'res2a_branch2b_zeropadding')
(12, u'res2a_branch2b')
(13, u'bn2a_branch2b')
(14, u'scale2a_branch2b')
(15, u'res2a_branch2b_relu')
(16, u'res2a_branch2c')
(17, u'res2a_branch1')
(18, u'bn2a_branch2c')
(19, u'bn2a_branch1')
(20, u'scale2a_branch2c')
(21, u'scale2a_branch1')
(22, u'res2a')
(23, u'res2a_relu')
(24, u'res2b_branch2a')
(25, u'bn2b_branch2a')
(26, u'scale2b_branch2a')
(27, u'res2b_branch2a_relu')
(28, u'res2b_branch2b_zeropadding')
(29, u'res2b_branch2b')
(30, u'bn2b_branch2b')
(31, u'scale2b_branch2b')
(32, u'res2b_branch2b_relu')
(33, u'res2b_branch2c')
(34, u'bn2b_branch2c')
(35, u'scale2b_branch2c')
(36, u'res2b')
(37, u'res2b_relu')
(38, u'res2c_branch2a')
(39, u'bn2c_branch2a')
(40, u'scale2c_branch2a')
(41, u'res2c_branch2a_relu')
(42, u'res2c

In [9]:
for layer in model.layers[:452]:
    layer.trainable = False
for layer in model.layers[452:]:
    layer.trainable = True

In [10]:
# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
from keras.optimizers import SGD
opt = SGD(lr=1e-3, decay=0.0, momentum=0.9, nesterov=True)
model.compile(optimizer=opt,
              loss="categorical_crossentropy",
              metrics=["accuracy"])


In [ ]:
from keras.callbacks import TensorBoard
tb = TensorBoard()

In [ ]:
num_epochs = 2


# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4, callbacks=[checkpointer])


Epoch 1/2
 89265/174074 [==============>...............] - ETA: 25193s - loss: 2.0797 - acc: 0.5711

In [ ]:
num_epochs = 2


# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4, callbacks=[checkpointer])


Epoch 1/2
174074/174074 [==============================] - ETA: 0s - loss: 2.5226 - acc: 0.5129 - val_loss: 2.5121 - val_acc: 0.5230
Epoch 2/2
174073/174074 [============================>.] - ETA: 0s - loss: 2.3817 - acc: 0.5312

In [4]:
for i,layer in enumerate(model.layers):
    print (i, layer.name)

(0, 'data')
(1, 'conv1_zeropadding')
(2, 'conv1')
(3, 'bn_conv1')
(4, 'scale_conv1')
(5, 'conv1_relu')
(6, 'pool1')
(7, 'res2a_branch2a')
(8, 'bn2a_branch2a')
(9, 'scale2a_branch2a')
(10, 'res2a_branch2a_relu')
(11, 'res2a_branch2b_zeropadding')
(12, 'res2a_branch2b')
(13, 'bn2a_branch2b')
(14, 'scale2a_branch2b')
(15, 'res2a_branch2b_relu')
(16, 'res2a_branch2c')
(17, 'res2a_branch1')
(18, 'bn2a_branch2c')
(19, 'bn2a_branch1')
(20, 'scale2a_branch2c')
(21, 'scale2a_branch1')
(22, 'res2a')
(23, 'res2a_relu')
(24, 'res2b_branch2a')
(25, 'bn2b_branch2a')
(26, 'scale2b_branch2a')
(27, 'res2b_branch2a_relu')
(28, 'res2b_branch2b_zeropadding')
(29, 'res2b_branch2b')
(30, 'bn2b_branch2b')
(31, 'scale2b_branch2b')
(32, 'res2b_branch2b_relu')
(33, 'res2b_branch2c')
(34, 'bn2b_branch2c')
(35, 'scale2b_branch2c')
(36, 'res2b')
(37, 'res2b_relu')
(38, 'res2c_branch2a')
(39, 'bn2c_branch2a')
(40, 'scale2c_branch2a')
(41, 'res2c_branch2a_relu')
(42, 'res2c_branch2b_zeropadding')
(43, 'res2c_branch2